In [7]:
import numpy as np

def softmax(Z):
 """
 Compute softmax values for each sets of scores in V.
 each column of V is a set of scores.
 Z: a numpy array of shape (N, C)
 return a numpy array of shape (N, C)
 """
 e_Z = np.exp(Z)
 A = e_Z / e_Z.sum(axis = 1, keepdims = True)
 return A

def softmax_stable(Z):
 """
 Compute softmax values for each sets of scores in Z.
 each row of Z is a set of scores.
 """
 # Z = Z.reshape(Z.shape[0], -1)
 e_Z = np.exp(Z - np.max(Z, axis = 1, keepdims = True))
 A = e_Z / e_Z.sum(axis = 1, keepdims = True)
 return A

def softmax_loss(X, y, W):
 """
 W: 2d numpy array of shape (d, C),
 each column correspoding to one output node
 X: 2d numpy array of shape (N, d), each row is one data point
 y: 1d numpy array -- label of each row of X
 """
 A = softmax_stable(X.dot(W))
 id0 = range(X.shape[0]) # indexes in axis 0, indexes in axis 1 are in y
 return -np.mean(np.log(A[id0, y]))

def softmax_grad(X, y, W):
 """
 W: 2d numpy array of shape (d, C),
 each column correspoding to one output node
 X: 2d numpy array of shape (N, d), each row is one data point
 y: 1d numpy array -- label of each row of X
 """
 A = softmax_stable(X.dot(W)) # shape of (N, C)
 id0 = range(X.shape[0])
 A[id0, y] -= 1 # A - Y, shape of (N, C)
 return X.T.dot(A)/X.shape[0]

def softmax_fit(X, y, W, lr = 0.01, nepoches = 100, tol = 1e-5, batch_size = 10):
 W_old = W.copy()
 ep = 0
 loss_hist = [softmax_loss(X, y, W)] # store history of loss
 N = X.shape[0]
 nbatches = int(np.ceil(float(N)/batch_size))
 while ep < nepoches:
  ep += 1
  mix_ids = np.random.permutation(N) # mix data
  for i in range(nbatches):
   # get the i-th batch
   batch_ids = mix_ids[batch_size*i:min(batch_size*(i+1), N)]
   X_batch, y_batch = X[batch_ids], y[batch_ids]
   W -= lr*softmax_grad(X_batch, y_batch, W) # update gradient descent
  loss_hist.append(softmax_loss(X, y, W))
  if np.linalg.norm(W - W_old)/W.size < tol:
   break
  W_old = W.copy()
 return W, loss_hist

def pred(W, X):
 """
 predict output of each columns of X . Class of each x_i is determined by
 location of max probability. Note that classes are indexed from 0.
 """
 return np.argmax(X.dot(W), axis =1)

C, N = 5, 500 # number of classes and number of points per class
means = [[2, 2], [8, 3], [3, 6], [14, 2], [12, 8]]
cov = [[1, 0], [0, 1]]
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X2 = np.random.multivariate_normal(means[2], cov, N)
X3 = np.random.multivariate_normal(means[3], cov, N)
X4 = np.random.multivariate_normal(means[4], cov, N)
X = np.concatenate((X0, X1, X2, X3, X4), axis = 0) # each row is a datapoint
Xbar = np.concatenate((X, np.ones((X.shape[0], 1))), axis = 1) # bias trick
y = np.asarray([0]*N + [1]*N + [2]*N+ [3]*N + [4]*N)
W_init = np.random.randn(Xbar.shape[1], C)
W, loss_hist = softmax_fit(Xbar, y, W_init, batch_size = 10, nepoches = 100, lr = 0.05)

In [ ]:
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
mnist = fetch_mldata('MNIST original', data_home='../../data/')
X = mnist.data
y = mnist.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000)
model = LogisticRegression(C = 1e5,
solver = 'lbfgs', multi_class = 'multinomial') # C is inverse of lam
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy %.2f %%" % (100*accuracy_score(y_test, y_pred.tolist())))